# Setup

In [28]:
OUTPUT_DIR = "/tmp/"

# Download Video

In [19]:
from freespeech.lib import youtube
import uuid


# VIDEO_URL = "https://youtu.be/WQlyNwp7meM"
VIDEO_URL = "https://www.youtube.com/watch?v=dnFk1_DM9QQ"
audio_file, video_file, meta, subtitles = youtube.download(
    VIDEO_URL,
    audio_filename=f"{str(uuid.uuid4())}.wav",
    video_filename=f"{str(uuid.uuid4())}.mp4",
    output_dir=OUTPUT_DIR)

KeyError: 'bitrate'

In [29]:
from freespeech.lib import media

audio_file = "/tmp/zel-un-final.wav"
audio_cut = await media.cut(audio_file, start="00:00:00", finish="00:03:08", output_dir=OUTPUT_DIR)

# Transcribe

In [31]:
from freespeech.lib import media, speech
from freespeech.lib.storage import obj

# audio_file = "/tmp/zel-un.wav"
audio_file = audio_cut

audio_file_mono = await media.multi_channel_audio_to_mono(audio_file, output_dir=OUTPUT_DIR)
(audio_info, *_), _ = media.probe(audio_file_mono)

audio_uri = f"gs://freespeech-tests/transcribe/{audio_file_mono.name}"
await obj.put(audio_file_mono, audio_uri)

transcript = await speech.transcribe(audio_uri, lang="en-US", provider="Azure")

In [30]:
events_1 = transcript

In [22]:
[e for e in transcript if e.time_ms is None]

[]

In [13]:
from dataclasses import replace


events = [replace(event, time_ms=(event.time_ms + 10 * 60 * 1000) if event.time_ms is not None else 0) for event in transcript]

In [24]:
transcript

[Event(time_ms=50, chunks=['Present a formula that can work not only for us, but for everyone who may find themselves in similar circumstances as we did.'], duration_ms=10840, voice=Voice(character='Ada', pitch=0.0, speech_rate=1.0)),
 Event(time_ms=11560, chunks=["It's a formula that punishes crime, protects life, restores security and territorial integrity, guarantees security and provides determination."], duration_ms=13560, voice=Voice(character='Ada', pitch=0.0, speech_rate=1.0)),
 Event(time_ms=25920, chunks=['There are five.'], duration_ms=1160, voice=Voice(character='Ada', pitch=0.0, speech_rate=1.0)),
 Event(time_ms=27720, chunks=['Break conditions for peace.'], duration_ms=1920, voice=Voice(character='Ada', pitch=0.0, speech_rate=1.0)),
 Event(time_ms=30960, chunks=['Yeah, Mr. President of the General Assembly, dear Secretary General of the United Nations, dear Heads of States and governments, dear journalists, nations of the world.'], duration_ms=13750, voice=Voice(character

In [32]:
from freespeech.lib import gdocs
from freespeech.types import Transcript


gdocs.create(Transcript(transcript, lang="en-US"), format="SSMD-LITE")

'https://docs.google.com/document/d/1QmguOS38oAgnXB_LjITOlbAfiuD9ZZObOJWEdwE4Qcg/edit#'

# Translate

In [ ]:
source_transcript = await notion.get_transcript(str(target_transcript.source))
translated_events = language.translate_events(
    source_transcript.events,
    source=source_transcript.lang,
    target=target_transcript.lang,
)

updated_transcript = await notion.put_transcript(
    database_id=database_id, transcript=replace(target_transcript, events=translated_events)
)

In [15]:
from freespeech.lib import transcript


transcript_text = """
language: en-US
format: SSMD-LITE

00:00:05@1.0 (Greta) Hello!
(Melinda) How are you? 
"""
transcript = transcript.parse_transcript(transcript_text)



In [16]:
from freespeech.lib import speech


res = await speech.synthesize_events(transcript.events, transcript.lang, "/tmp/")

In [17]:
audio, voices = res

In [22]:
import IPython.display as ipd


ipd.Audio(filename=audio_cut)